# Assignment 6

### Part A

In [22]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [23]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Price in 1 Year  Probability
0               32          0.3
1               43          0.4
2               54          0.3

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [24]:
def exp_return(data, P):
    # Write your code here
    # exp_return(data, 40) should output 0.07500000000000002
    df = data.copy()
    df['Return'] = (data['Price in 1 Year'] - P) / P
    df['Expected_Return'] = df['Return'] * data['Probability']
    exp_ret = sum(df['Expected_Return'])

    return exp_ret

#exp_return(data, 40)

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [25]:
def volatility(data, P):
    # Write your code here
    # volatility(data,40) should output 0.21301408404140795
    
    df = data.copy()
    exp_ret = exp_return(data, P)
    df['Return'] = (data['Price in 1 Year'] - P) / P

    df['Variance'] = df['Probability'] * ((df['Return'] - exp_ret) ** 2)

    SD = np.sqrt(sum(df['Variance']))

    return SD

#volatility(data,40)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [26]:
def interval(data, P):
    # Write your code here
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]
    df = data.copy()
    exp_ret = exp_return(data, P)
    SD = volatility(data, P)
    z_score = 1.96

    SE = z_score * SD / np.sqrt(len(df))
    ret = [exp_ret - SE, exp_ret + SE]
    
    return ret

#interval(data, 40)

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [27]:
def exp_return_capm(beta, market_return, rf):
    # Write your code here
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028

    exp_ret = rf + beta *(market_return - rf)

    return exp_ret

#exp_return_capm(-0.6,0.1, 0.02)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [28]:
def clean_data(path):
    # Write your code here
    # This should yield data with 403316 rows × 4 columns
    df = pd.read_csv(path)

    df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
    df = df.dropna(subset=['RET'])

    df = df.groupby('CUSIP').filter(lambda x: len(x) >= 500)

    return df

#clean_data('monthlycrsp_mod6.csv')


Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [29]:
def avg_return(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns
    df = data.copy()
    average_ret = df.groupby('CUSIP')['RET'].mean()
    average_ret = average_ret.reset_index()
    average_ret.columns = ['CUSIP', 'Average RET']

    # output = pd.DataFrame(data = {
    #     "CUSIP" : [],
    #     "Average Return" : []
    # })

    output = average_ret
    
    return output
#avg_return(clean_data('monthlycrsp_mod6.csv'))


In [30]:
def security_volatilities(data):
    # Write your code here
    # This should yield data with 679 rows × 2 columns

    df = data.copy()
    average_ret = df.groupby('CUSIP')['RET'].std()
    average_ret = average_ret.reset_index()
    average_ret.columns = ['CUSIP', 'Volatility']
    
    # output = pd.DataFrame(data = {
    #     "CUSIP" : [],
    #     "Volatility" : []
    # })
    output = average_ret
    
    return output

#security_volatilities(clean_data('monthlycrsp_mod6.csv'))

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [31]:
def correlation(ret_data, vol_data):
    # Write your code here
    # correlation(rets, vols) should yield 0.44837090727059614

    df_merged = pd.merge(ret_data, vol_data, on='CUSIP', how='inner')
    corr = df_merged['Average RET'].corr(df_merged['Volatility'])
    
    return corr

#correlation(avg_return(clean_data('monthlycrsp_mod6.csv')), security_volatilities(clean_data('monthlycrsp_mod6.csv')))